## 1. Preprocesamiento de Datos

### Conversión de variables categóricas:
- Convertir directamente las variables categóricas a numéricas.
- Utilizar **Label Encoding** o **OneHotEncoding**.
- Representar las variables categóricas utilizando la probabilidad de la clase objetivo.

### Imputación de valores faltantes:
- Imputar los valores faltantes utilizando el valor **0**.
- Imputar los valores faltantes utilizando la **media** para variables numéricas.


In [17]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold

from xgboost import XGBClassifier
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score


In [18]:
# Cargar los datos
# train_path = '/home/alvaro/Documentos/DATA/titanic/train.csv'
# test_path = '/home/alvaro/Documentos/DATA/titanic/test.csv'
# windows
train_path = 'C:/Users/alvar/Documents/train.csv'
test_path = 'C:/Users/alvar/Documents/test.csv'

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

# ======== PREPROCESAMIENTO DE TRAIN ========

# Separar la variable objetivo
supervivientes_col = train["Survived"].copy()

# Eliminar columnas innecesarias
train.drop(columns=["Survived", "Cabin", "Name", "Ticket"], inplace=True)

# Imputadores entrenados con TRAIN
num_cols = train.select_dtypes(include=['int64', 'float64']).columns
cat_cols = train.select_dtypes(include='object').columns

num_imputer = SimpleImputer(strategy='mean')
cat_imputer = SimpleImputer(strategy='constant', fill_value='missing')

train[num_cols] = num_imputer.fit_transform(train[num_cols])
train[cat_cols] = cat_imputer.fit_transform(train[cat_cols])

# One-hot encoding
cols_to_encode = ['Sex', 'Embarked']
train = pd.get_dummies(train, columns=cols_to_encode)

# Guardamos las columnas resultantes para replicar en test
train_columns = train.columns

# Volver a añadir la variable objetivo
train["Survived"] = supervivientes_col



# ======== PREPROCESAMIENTO DE TEST ========

# Eliminar las mismas columnas
test.drop(columns=["Cabin", "Name", "Ticket"], inplace=True)

# Asegurar que estén las mismas columnas para imputar
test[num_cols] = num_imputer.transform(test[num_cols])
test[cat_cols] = cat_imputer.transform(test[cat_cols])

# One-hot encoding
test = pd.get_dummies(test, columns=cols_to_encode)

# Añadir columnas faltantes (por si test no tiene todas las categorías)
for col in train_columns:
    if col not in test.columns and col != "Survived":
        test[col] = 0

# Reordenar columnas para que coincidan
test = test[[col for col in train_columns if col != "Survived"]]



# Listo para usar en predicciones
print("Train shape:", train.shape)
print("Test shape:", test.shape)


Train shape: (891, 13)
Test shape: (418, 12)


In [19]:
train

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Embarked_missing,Survived
0,1.0,3.0,22.000000,1.0,0.0,7.2500,False,True,False,False,True,False,0
1,2.0,1.0,38.000000,1.0,0.0,71.2833,True,False,True,False,False,False,1
2,3.0,3.0,26.000000,0.0,0.0,7.9250,True,False,False,False,True,False,1
3,4.0,1.0,35.000000,1.0,0.0,53.1000,True,False,False,False,True,False,1
4,5.0,3.0,35.000000,0.0,0.0,8.0500,False,True,False,False,True,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887.0,2.0,27.000000,0.0,0.0,13.0000,False,True,False,False,True,False,0
887,888.0,1.0,19.000000,0.0,0.0,30.0000,True,False,False,False,True,False,1
888,889.0,3.0,29.699118,1.0,2.0,23.4500,True,False,False,False,True,False,0
889,890.0,1.0,26.000000,0.0,0.0,30.0000,False,True,True,False,False,False,1


## 2. Selección de Características

Explorar diferentes métodos de selección de características:

- Filtrado hi-cuadrado.
- Embebido RFE.
- Envolvente regresión logística.


In [20]:
# CHI2

# X: todas las columnas menos la target
X = train.drop(columns=["Survived"])
y = train["Survived"]

# Selección con Chi-cuadrado
selector_chi2 = SelectKBest(score_func=chi2, k=5)  # elige las 10 mejores
X_chi2 = selector_chi2.fit_transform(X, y)

# Ver columnas seleccionadas
selected_cols_chi2 = X.columns[selector_chi2.get_support()]
print("Características seleccionadas con Chi-cuadrado:")
print(selected_cols_chi2)


Características seleccionadas con Chi-cuadrado:
Index(['Pclass', 'Age', 'Fare', 'Sex_female', 'Sex_male'], dtype='object')


In [21]:
# RFE

# Usamos un Random Forest como modelo base
model = RandomForestClassifier(random_state=42)
selector_rfe = RFE(estimator=model, n_features_to_select=5)  # elige 10 features

X_rfe = selector_rfe.fit_transform(X, y)
selected_cols_rfe = X.columns[selector_rfe.get_support()]
print("Características seleccionadas con RFE:")
print(selected_cols_rfe)


Características seleccionadas con RFE:
Index(['PassengerId', 'Age', 'Fare', 'Sex_female', 'Sex_male'], dtype='object')


In [22]:
# Envolvente Regresion Logistica

# Modelo base: regresión logística
logreg = LogisticRegression(max_iter=2000, solver='liblinear')


# Cross-validation
cv = StratifiedKFold(n_splits=5)

selector_rfecv = RFECV(estimator=logreg, step=1, cv=cv, scoring='accuracy')
X_rfecv = selector_rfecv.fit_transform(X, y)

selected_cols_rfecv = X.columns[selector_rfecv.get_support()]
print("Características seleccionadas con RFECV (envolvente con regresión logística):")
print(selected_cols_rfecv)


Características seleccionadas con RFECV (envolvente con regresión logística):
Index(['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_female',
       'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S',
       'Embarked_missing'],
      dtype='object')


## 3. Evaluación de Modelos con Validación Cruzada
Evaluar para la combinación de cada una de las estrategias de codificación
de categorías, imputación y selección de características

- Implementar al menos **dos modelos predictivos**, por ejemplo:
  - **Random Forest**
  - **XGBoost**

- Utilizar **validación cruzada de 10 folds** para evaluar el rendimiento de cada modelo.

- Obtener las siguientes **métricas de desempeño**:
  - Precisión
  - Exactitud
  - Recall
  - F1-Score


In [23]:
#['Pclass', 'Age', 'Fare', 'Sex_female', 'Sex_male']
# === APARTADO 3: Evaluación de Modelos con Validación Cruzada ===

# Preparar los datos y modelos
X = train.drop(columns=["Survived"])
y = train["Survived"]

# Características seleccionadas anteriormente
X_chi2 = X[selected_cols_chi2]
X_rfe = X[selected_cols_rfe]

# Modelos a evaluar
modelos = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
}

# Métricas personalizadas
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score)
}

# Validación cruzada estratificada con 10 folds
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Evaluar cada combinación
resultados = {}

for nombre_modelo, modelo in modelos.items():
    for nombre_selector, X_sel in [("Chi2", X_chi2), ("RFE", X_rfe)]:
        print(f"Evaluando {nombre_modelo} con selección {nombre_selector}...")
        cv_result = cross_validate(modelo, X_sel, y, cv=cv, scoring=scoring)
        
        resumen = {metric: (np.mean(cv_result[f'test_{metric}']), np.std(cv_result[f'test_{metric}']))
                   for metric in scoring.keys()}
        
        resultados[f"{nombre_modelo}_{nombre_selector}"] = resumen

# Mostrar resultados
for clave, metricas in resultados.items():
    print(f"\nResultados para {clave}:")
    for metrica, (media, std) in metricas.items():
        print(f"{metrica.capitalize()}: {media:.4f} (±{std:.4f})")


Evaluando RandomForest con selección Chi2...
Evaluando RandomForest con selección RFE...
Evaluando XGBoost con selección Chi2...


c:\Users\alvar\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [12:50:17] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\alvar\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [12:50:18] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\alvar\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [12:50:18] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\alvar\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [12:50:18] WARNING: C:\actio

Evaluando XGBoost con selección RFE...


c:\Users\alvar\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [12:50:18] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\alvar\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [12:50:18] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\alvar\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [12:50:18] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\alvar\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [12:50:18] WARNING: C:\actio


Resultados para RandomForest_Chi2:
Accuracy: 0.8305 (±0.0369)
Precision: 0.7929 (±0.0430)
Recall: 0.7571 (±0.0893)
F1: 0.7720 (±0.0575)

Resultados para RandomForest_RFE:
Accuracy: 0.7867 (±0.0253)
Precision: 0.7361 (±0.0429)
Recall: 0.6956 (±0.0602)
F1: 0.7137 (±0.0398)

Resultados para XGBoost_Chi2:
Accuracy: 0.8181 (±0.0242)
Precision: 0.7794 (±0.0282)
Recall: 0.7366 (±0.0821)
F1: 0.7545 (±0.0433)

Resultados para XGBoost_RFE:
Accuracy: 0.7901 (±0.0286)
Precision: 0.7475 (±0.0489)
Recall: 0.6901 (±0.0668)
F1: 0.7152 (±0.0417)


## 4. Almacenamiento de Resultados en MySQL

Diseñar y crear una base de datos en **MySQL** para almacenar los resultados de las evaluaciones.  
Se recomienda incluir las siguientes tablas:

- **Información del preprocesamiento**: (estrategia de imputación, codificación utilizada, etc.).

- **Métricas de desempeño por modelo y por fold**  

- **Resumen general de la evaluación**: (por ejemplo, promedio y desviación estándar de las métricas).


In [31]:
# === APARTADO 4: Almacenamiento de Resultados en MySQL ===

import mysql.connector

# Conexión a la base de datos MySQL
conexion = mysql.connector.connect(
    user="alv_mar",
    password="16_alv_mar_93",
    host="82.165.173.36",
    port="31234"
)

cursor = conexion.cursor()
cursor.execute("USE bd_alv_mar_graf2")

# Crear las tablas
cursor.execute("""
CREATE TABLE IF NOT EXISTS preprocesamiento (
    id INT AUTO_INCREMENT PRIMARY KEY,
    codificacion VARCHAR(50),
    imputacion_num VARCHAR(50),
    imputacion_cat VARCHAR(50),
    seleccion_caracteristicas VARCHAR(50)
)
""")
cursor.execute("""
CREATE TABLE IF NOT EXISTS metricas_fold (
    id INT AUTO_INCREMENT PRIMARY KEY,
    modelo VARCHAR(50),
    seleccion VARCHAR(50),
    fold INT,
    accuracy FLOAT,
    precision_score FLOAT,
    recall FLOAT,
    f1 FLOAT
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS resumen_evaluacion (
    id INT AUTO_INCREMENT PRIMARY KEY,
    modelo VARCHAR(50),
    seleccion VARCHAR(50),
    metric VARCHAR(20),
    media FLOAT,
    desviacion FLOAT
)
""")
conexion.commit()

# Insertar información de preprocesamiento
preprocesamiento_info = [
    ("OneHotEncoding", "media", "missing", "Chi2"),
    ("OneHotEncoding", "media", "missing", "RFE")
]
for fila in preprocesamiento_info:
    cursor.execute("INSERT INTO preprocesamiento (codificacion, imputacion_num, imputacion_cat, seleccion_caracteristicas) VALUES (%s, %s, %s, %s)", fila)

# Insertar métricas por fold y resumen
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

for nombre_modelo, modelo in modelos.items():
    for nombre_selector, X_sel in [("Chi2", X_chi2), ("RFE", X_rfe)]:
        print(f"Ingresando a MySQL: {nombre_modelo} + {nombre_selector}")
        # Validación cruzada manual para registrar fold a fold
        i = 1
        for train_idx, test_idx in cv.split(X_sel, y):
            modelo.fit(X_sel.iloc[train_idx], y.iloc[train_idx])
            y_pred = modelo.predict(X_sel.iloc[test_idx])
            acc = accuracy_score(y.iloc[test_idx], y_pred)
            pre = precision_score(y.iloc[test_idx], y_pred)
            rec = recall_score(y.iloc[test_idx], y_pred)
            f1s = f1_score(y.iloc[test_idx], y_pred)
            cursor.execute(
                "INSERT INTO metricas_fold (modelo, seleccion, fold, accuracy, precision_score, recall, f1) VALUES (%s, %s, %s, %s, %s, %s, %s)",
                (nombre_modelo, nombre_selector, i, acc, pre, rec, f1s)
            )
            i += 1

        # Insertar resumen
        for metrica, (media, std) in resultados[f"{nombre_modelo}_{nombre_selector}"].items():
            cursor.execute(
                "INSERT INTO resumen_evaluacion (modelo, seleccion, metric, media, desviacion) VALUES (%s, %s, %s, %s, %s)",
                (nombre_modelo, nombre_selector, metrica, media, std)
            )

conexion.commit()
cursor.close()
conexion.close()
print("✅ Resultados almacenados correctamente en MySQL.")


Ingresando a MySQL: RandomForest + Chi2
Ingresando a MySQL: RandomForest + RFE
Ingresando a MySQL: XGBoost + Chi2


c:\Users\alvar\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [12:54:56] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\alvar\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [12:54:56] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\alvar\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [12:54:56] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\alvar\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [12:54:57] WARNING: C:\actio

Ingresando a MySQL: XGBoost + RFE


c:\Users\alvar\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [12:54:57] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\alvar\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [12:54:57] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\alvar\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [12:54:58] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\alvar\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [12:54:58] WARNING: C:\actio

✅ Resultados almacenados correctamente en MySQL.


In [ ]:
# Listar tablas
conexion = mysql.connector.connect(
    user="alv_mar",
    password="16_alv_mar_93",
    host="82.165.173.36",
    port="31234"
)

cursor = conexion.cursor()
cursor.execute("USE bd_alv_mar_graf2")

cursor.execute("SHOW TABLES;")
tablas = cursor.fetchall()

print("Tablas en la base de datos:")
for tabla in tablas:
    print(f"- {tabla[0]}")
    
cursor.execute("SELECT * FROM resumen_evaluacion;")
resultados = cursor.fetchall()

print("Contenido de la tabla metricas_fold:")
for fila in resultados:
    print(fila)

cursor.close()
conexion.close()

Tablas en la base de datos:
- metricas_fold
- preprocesamiento
- resumen_evaluacion
Contenido de la tabla metricas_fold:
(1, 'RandomForest', 'Chi2', 'accuracy', 0.830474, 0.0369107)
(2, 'RandomForest', 'Chi2', 'precision', 0.792945, 0.0430236)
(3, 'RandomForest', 'Chi2', 'recall', 0.757143, 0.0892855)
(4, 'RandomForest', 'Chi2', 'f1', 0.772011, 0.0574932)
(5, 'RandomForest', 'RFE', 'accuracy', 0.786729, 0.0253215)
(6, 'RandomForest', 'RFE', 'precision', 0.736093, 0.0429021)
(7, 'RandomForest', 'RFE', 'recall', 0.69563, 0.0601563)
(8, 'RandomForest', 'RFE', 'f1', 0.71365, 0.0398385)
(9, 'XGBoost', 'Chi2', 'accuracy', 0.81814, 0.0242403)
(10, 'XGBoost', 'Chi2', 'precision', 0.779435, 0.0282028)
(11, 'XGBoost', 'Chi2', 'recall', 0.736639, 0.0821342)
(12, 'XGBoost', 'Chi2', 'f1', 0.754487, 0.0432685)
(13, 'XGBoost', 'RFE', 'accuracy', 0.7901, 0.0285964)
(14, 'XGBoost', 'RFE', 'precision', 0.747483, 0.0489495)
(15, 'XGBoost', 'RFE', 'recall', 0.690084, 0.0668136)
(16, 'XGBoost', 'RFE', 'f1'